In [1]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install bs4

'apt' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.


  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=11b4268a40a7edca9811901b06a3cfd30899c1bc3e6c656f249936780694a5e3
  Stored in directory: c:\users\rida\appdata\local\pip\cache\wheels\73\2b\cb\099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import math

In [2]:
import json
import numpy as np
import pandas as pd       
import ctypes

In [3]:
from selenium import webdriver   
import os
driver=webdriver.Chrome("C:\\Users\\Rida\\Downloads\\chromedriver.exe")




In [4]:
#Nebraska= pd.read_csv(r'C:\Users\Rida\Desktop\GRA\week 8\Institutions\Nebraska.csv')
#Nevada= pd.read_csv(r'C:\Users\Rida\Desktop\GRA\week 8\Institutions\Nevada.csv')
#NewHampshire= pd.read_csv(r'C:\Users\Rida\Desktop\GRA\week 8\Institutions\New Hampshire.csv')
#NewJersey= pd.read_csv(r'C:\Users\Rida\Desktop\GRA\week 8\Institutions\New Jersey.csv')
#NewYork=pd.read_csv(r'C:\Users\Rida\Desktop\GRA\week 8\Institutions\New York.csv')
#NorthCarolina= pd.read_csv(r'C:\Users\Rida\Desktop\GRA\week 8\Institutions\North Carolina.csv')
#NorthDakota= pd.read_csv(r'C:\Users\Rida\Desktop\GRA\week 8\Institutions\North Dakota.csv')

#Ohio = pd.read_csv(r'C:\Users\Rida\Desktop\GRA\week 8\Institutions\Ohio.csv')

Texas3 = pd.read_csv(r'C:\Users\Rida\Desktop\CrosswalkRawData2\South Carolina.csv')


 
states = [Texas3]
state_name = ['South Carolina']

In [5]:
for j in range(len(states)):
  print('----------',state_name[j],'----------')
  hospitals = states[j]
  #hospitals = hospitals[hospitals['AMC_No'].notna()]
  #hospitals['AMC_No'] = hospitals["AMC_No"].astype(int)

  baseurl = 'https://freida.ama-assn.org/institution/'
  URLS = []
  for index, i in hospitals.iterrows():
    if hospitals['AMC_No'][j] < 100000:
      print(hospitals['AMC_No'][j], ' is less than 100000.')
      URLS.append(str(baseurl + '0' + str(i['AMC_No'])))
      print(URLS[j])
    else:
      URLS.append(str(baseurl + str(i['AMC_No'])))
  hospitals['URLS'] = URLS
  print(len(hospitals), hospitals.head())
  def driversetup():
      options = webdriver.ChromeOptions()
      #run Selenium in headless mode
      options.add_argument('--headless')
      options.add_argument('--no-sandbox')
      options.add_argument("enable-automation")
      #overcome limited resource problems
      options.add_argument('--disable-dev-shm-usage')
      options.add_argument("lang=en")
      #open Browser in maximized mode
      options.add_argument("start-maximized")
      #disable infobars
      options.add_argument("disable-infobars")
      #disable extension
      options.add_argument("--disable-browser-side-navigation")
      options.add_argument("--disable-gpu")
      options.add_argument("--disable-extensions")
      options.add_argument("--incognito")
      options.add_argument("--disable-blink-features=AutomationControlled")
      driver = webdriver.Chrome(options=options)
      driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")
      return driver

  def pagesource(url, driver):
      driver = driver
      driver.get(url)
      time.sleep(12)
      soup = BeautifulSoup(driver.page_source)
      return soup
  
  mc = []
  grad = []
  mcl = []
  add = []
  d = driversetup()
  for index, i in hospitals.iterrows():
    url = i['URLS']
    s = pagesource(url, d)
    med_school = list(s.findAll('small', class_="bold"))
    address = list(s.findAll('div', class_="details__address ng-star-inserted")) 
    if len(med_school) == 0:
      mc.append("")
      mcl.append("")
      grad.append("")
    elif len(med_school) == 1:
      a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
      name = a[0].findAll('small', class_='bold')[0].text
      relationship = a[0].findAll('div', class_='')[0].text
      if 'Limited' in relationship:
        mcl.append(name)
        mc.append("")
        grad.append("")
      elif 'Major' in relationship:
        mc.append(name)
        mcl.append("")
        grad.append("")
      elif 'Graduate' in relationship:
        mc.append("")
        mcl.append("")
        grad.append(name)
      else:
        print("NEW RELATIONSHIP FOUND : ", name)
        break
    elif len(med_school) > 1:   
        major = []
        limited = []
        graduate = []
        a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
        for i in range(len(a)):
          name = a[i].findAll('small', class_='bold')[0].text
          relationship = a[i].findAll('div', class_='')[0].text
          if 'Limited' in relationship:
            limited.append(name)
          elif 'Graduate' in relationship:
            graduate.append(name)
          elif 'Major' in relationship:
            major.append(name)
          else:
            print("New relationship found")
            break 
        if major == []:
          major = ''
        if limited == []:
          limited = ''
        if graduate == []:
          graduate = ''
        mc.append(major)
        mcl.append(limited)
        grad.append(graduate)
    if len(address) != 0:
      address[0].br.string = ', '
      add.append(address[0].text)
    else:
      add.append("")
    print(index, 'mc', len(mc), mc[index], 'mcl', len(mcl), mcl[index], 'grad', len(grad), grad[index], 'add', len(add), add[index])
    if len(mc) > (index+1):
      break
    
  print(len(mc),len(mcl),len(grad),len(add))
  hospitals['Affiliation_Major'] = mc
  hospitals['Affiliation_Limited'] = mcl
  hospitals['Affiliation_Graduate'] = grad
  hospitals['Medical_Center_Address'] = add 
  filename = state_name[j] + '_crosswalk.csv'
  hospitals.to_csv(filename)




---------- South Carolina ----------
141    AMC_No                          AMC_Name  \
0  459556         Advanced Cosmetic Surgery   
1  459559    Aiken Regional Medical Centers   
2  450732                      AnMed Health   
3  450055   Atlantic Coast Pain Specialists   
4  458041     Berkeley County Probate Court   

                                             URLS  
0  https://freida.ama-assn.org/institution/459556  
1  https://freida.ama-assn.org/institution/459559  
2  https://freida.ama-assn.org/institution/450732  
3  https://freida.ama-assn.org/institution/450055  
4  https://freida.ama-assn.org/institution/458041  
0 mc 1  mcl 1  grad 1  add 1 29 Rocky Slope Rd, Greenville, SC 29607
1 mc 2  mcl 2 Edward Via Coll of Osteopathic Med-Carolinas grad 2  add 2 302 University Pkwy, Aiken, SC 29801
2 mc 3 ['Edward Via Coll of Osteopathic Med-Carolinas'] mcl 3 ['Univ of South Carolina Sch of Med Columbia'] grad 3  add 3 800 N Fant St, Anderson, SC 29621
3 mc 4  mcl 4  grad 4  add 4

85 mc 86  mcl 86  grad 86  add 86 400 Palmetto Health Pkwy, Columbia, SC 29212
86 mc 87 Univ of South Carolina Sch of Med Greenville mcl 87  grad 87  add 87 830 S Buncombe Rd, Greer, SC 29650
87 mc 88 Univ of South Carolina Sch of Med Greenville mcl 88  grad 88  add 88 701 Grove Rd, Greenville, SC 29605
88 mc 89 Univ of South Carolina Sch of Med Greenville mcl 89  grad 89  add 89 298 Memorial Dr, Seneca, SC 29672
89 mc 90 Univ of South Carolina Sch of Med Columbia mcl 90  grad 90  add 90 5 Richland Medical Pk, Columbia, SC 29203
90 mc 91  mcl 91 Edward Via Coll of Osteopathic Med-Carolinas grad 91  add 91 129 N Washington St, Sumter, SC 29150
91 mc 92  mcl 92  grad 92  add 92 200 Patewood Dr, Greenville, SC 29615
92 mc 93 ['Univ of South Carolina Sch of Med Greenville'] mcl 93 ['Midwestern Univ Arizona Coll of Osteopathic Med', 'Med Coll of Georgia at Augusta Univ', 'Univ of South Carolina Sch of Med Columbia', 'West Virginia Sch of Osteopathic Med', 'California Northstate Univ Coll of

In [11]:
hospitals.to_csv(file)
filename = state_name[0] + '_crosswalk.csv'
hospitals.to_csv(filename)

NameError: name 'file' is not defined

In [ ]:
len(hospitals)

191

In [ ]:
# def driversetup():
#     options = webdriver.ChromeOptions()
#     #run Selenium in headless mode
#     options.add_argument('--headless')
#     options.add_argument('--no-sandbox')
#     options.add_argument("enable-automation")
#     #overcome limited resource problems
#     options.add_argument('--disable-dev-shm-usage')
#     options.add_argument("lang=en")
#     #open Browser in maximized mode
#     options.add_argument("start-maximized")
#     #disable infobars
#     options.add_argument("disable-infobars")
#     #disable extension
#     options.add_argument("--disable-browser-side-navigation")
#     options.add_argument("--disable-gpu")
#     options.add_argument("--disable-extensions")
#     options.add_argument("--incognito")
#     options.add_argument("--disable-blink-features=AutomationControlled")
    
#     driver = webdriver.Chrome(options=options)

#     driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

#     return driver

# def pagesource(url, driver):
#     driver = driver
#     driver.get(url)
#     time.sleep(12)
#     soup = BeautifulSoup(driver.page_source)
#     #driver.close()
#     return soup



In [ ]:
# d = driversetup()
# relationship = []
# for index, i in hospitals.iterrows():
#   url = i['URLS']
#   s = pagesource(url, d)
#   med_school = list(s.findAll('small', class_="bold"))
#   print(i.Med_center, index, relationship, len(med_school))
#   for i in range(len(med_school)):
#     a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
    
#     for i in range(len(a)):
#       rel = a[i].findAll('div', class_='')[0].text
#       if rel not in relationship:
#         print(med_school[i].text, rel)
#         relationship.append(rel)
# relationship

In [ ]:
# mc = []
# grad = []
# mcl = []
# add = []
# d = driversetup()
# for index, i in hospitals.iterrows():
#   url = i['URLS']
#   s = pagesource(url, d)
#   med_school = list(s.findAll('small', class_="bold"))
#   address = list(s.findAll('div', class_="details__address ng-star-inserted")) 
#   if len(med_school) == 0:
#     mc.append("")
#     mcl.append("")
#     grad.append("")
#   elif len(med_school) == 1:
#     a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
#     name = a[0].findAll('small', class_='bold')[0].text
#     relationship = a[0].findAll('div', class_='')[0].text
#     if 'Limited' in relationship:
#       mcl.append(name)
#       mc.append("")
#       grad.append("")
#     elif 'Major' in relationship:
#       mc.append(name)
#       mcl.append("")
#       grad.append("")
#     elif 'Graduate' in relationship:
#       mc.append("")
#       mcl.append("")
#       grad.append(name)
#     else:
#       print("NEW RELATIONSHIP FOUND : ", name)
#       break
#   elif len(med_school) > 1:   
#       major = []
#       limited = []
#       graduate = []
#       a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
#       for i in range(len(a)):
#         name = a[i].findAll('small', class_='bold')[0].text
#         relationship = a[i].findAll('div', class_='')[0].text
#         if 'Limited' in relationship:
#           limited.append(name)
#         elif 'Graduate' in relationship:
#           graduate.append(name)
#         elif 'Major' in relationship:
#           major.append(name)
#         else:
#           print("New relationship found")
#           break 
#       if major == []:
#         major = ''
#       if limited == []:
#         limited = ''
#       if grad == []:
#         grad = ''
#       mc.append(major)
#       mcl.append(limited)
#       grad.append(graduate)
#   if len(address) != 0:
#     address[0].br.string = ', '
#     add.append(address[0].text)
#   else:
#     add.append("")
#   print(index, 'mc', len(mc), mc[index], 'mcl', len(mcl), mcl[index], 'grad', len(grad), grad[index], 'add', len(add), add[index])
#   if len(mc) > (index+1):
#     break



In [ ]:
# print(len(mc),len(mcl),len(grad),len(add))
# hospitals['Affiliation_Major'] = mc
# hospitals['Affiliation_Limited'] = mcl
# hospitals['Affiliation_Graduate'] = grad
# hospitals['Medical_Center_Address'] = add

In [ ]:
# hospitals.to_csv('Alaska_crosswalk.csv')